***
# <font color=royalblue size=10>INTRODUÇÃO</font>

***
- Arquivo	: notebook.ipynb
- Título	: Exercício 5 e 6 - Disciplina de Redes Neurais Artificiais (DELT/UFMG)
- Autor	    : Gustavo Augusto Ortiz de Oliveira (gstvortiz@hotmail.com) <br> <br> <br>
- Descrição: A atividade dessa semana introduz uma arquitetura amplamente conhecida como Multi Layer Perceptron, ou MLP. As MLPs são estruturas de Redes Neurais Artificiais com duas ou mais camadas alimentadas para frente, tipicamente utilizando funções de ativação sigmoidal na camada intermediária. Diferentemente das estruturas apresentadas anteriormente, a rede não apresenta treinamentos a priori na camada escondida, necessitando de um algoritmo para ajuste simultâneo de pesos. O algoritmo mais difundido para o treinamento de uma rede MLP é chamado backpropagation, ele utiliza o conceito da retropropagação do erro da rede para corrigir os pesos da última camada até a primeira.


<div align = 'center'>
    <img src = "data/mlp.png">
</div>

***
# <font color=indianred size=10>BACKPROPAGATION</font>

***

## [1] A EQUAÇÃO DE AJUSTE

Para realizar a dedução das equações da retropropagação de erros, vamos utilizar o gradiente descendente em relação à função de erro de saída, conforme Equação 1, em que m é a quantidade de neurônios na camada de saída e k indica que o erro está sendo calculado para cada amostra do conjunto de dados de entrada. Esta notação será omitida por enquanto e será retomada ao final das deduções.


#### $$ \epsilon = \frac{1}{2}\sum^m_{j=1}(y_{j}-\hat{y}_{j})^2 $$

## [2] CAMADA DE SAÍDA


#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = \frac{\partial (\frac{1}{2}\sum^m_{j=1}(y_{j}-\hat{y}_{j})^2)}{\partial w_{ij}} = \frac{1}{2}\frac{\partial}{\partial w_{ij}}(y_{j}-\hat{y}_{j})^2 $$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = \frac{1}{2}2(y_j - \hat{y}_{j})\frac{\partial}{\partial w_{ij}}(y_j - \hat{y}_{j}) $$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = \epsilon_j \frac{\partial}{\partial w_{ij}}(y_j - g(u_{j})) $$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = -\epsilon_j\frac{\partial g(u_{j})}{\partial u_{j}}\frac{\partial u_{j}}{\partial w_{ij}} $$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = -\epsilon_j g'(u_{j})\frac{\partial u_{j}}{\partial w_{ij}}$$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = -\delta_j\frac{\partial u_{j}}{\partial w_{ij}}$$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = -\delta_j\frac{\partial (h(u)*w_j)}{\partial w_{ij}}$$

#### $$ \frac{\partial \epsilon}{\partial w_{ij}} = -\delta_jh(u_i)$$


Caminhando no sentido contrário do gradiente, temos:

#### $$ \Delta w_{ij}= \eta \delta_jh(u_i)$$


## [3] CAMADA ESCONDIDA

#### $$ \frac{\partial \epsilon}{\partial z_{li}} = \frac{\partial (\frac{1}{2}\sum^m_{j=1}(y_{j}-\hat{y}_{j})^2)}{\partial z_{li}} = \frac{1}{2}\frac{\partial}{\partial z_{li}}\sum^m_{j=1}(y_{j}-\hat{y}_{j})^2 $$

#### $$ \frac{\partial \epsilon_j}{\partial z_{li}} = \frac{1}{2}\frac{\partial}{\partial z_{li}}(y_{j}-\hat{y}_{j})^2 $$

#### $$ \frac{\partial \epsilon_j}{\partial z_{ij}} = -\delta_j\frac{\partial u_{j}}{\partial z_{ij}}$$

#### $$ \frac{\partial \epsilon_j}{\partial z_{ij}} = - \delta_{j} \frac{\partial (h(u)*w_j)}{\partial z_{li}} $$ 


#### $$ \frac{\partial \epsilon_j}{\partial z_{ij}} = - \delta_{j} w_{ij} \frac{\partial (h(u_{i}))}{\partial z_{li}}  $$ 

#### $$ \frac{\partial \epsilon_j}{\partial z_{ij}}  = - \delta_{j} w_{ij} h'({u_{i}})\frac{\partial (x*z_i)}{\partial z_{li}}$$ 

#### $$ \frac{\partial \epsilon_j}{\partial z_{ij}}  = - \delta_{j} w_{ij} h'({u_{i}})x_l$$ 

#### $$ \frac{\partial \epsilon}{\partial z_{li}} = - (\sum_{j=1}^{m} \delta_{j} w_{ij}) \space h'({u_{i}}) \space x_l$$ 

Caminhando no sentindo contrário do gradiente, temos:

#### $$ \Delta z_{li} = \eta (\sum_{j=1}^{m} \delta_{j} w_{ij}) \space h'({u_{i}}) \space x_l$$ 

***
# <font color=seagreen size=10>IMPLEMENTAÇÃO</font>

***

## [1] BIBLIOTECAS

In [ ]:
import os
import shutil
import datetime
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold

## [2] FUNÇÕES AUXILIARES

In [ ]:
def bias(m):
    m = np.array(m)
    if m.ndim > 1:
        return np.hstack((m, np.ones((m.shape[0], 1))))
    else:
        return np.append(m, 1)
    
def sech2(x):
    return 2 / (np.exp(x) + np.exp(-x))**2

def score(y_true, y_pred):
    comp = (y_true.ravel() == y_pred.ravel())
    return sum(comp) / len(comp)

## [3] CLASSE DA REDE

In [ ]:
class TwoLayersAdaline:
    def __init__(self, p = 100, η = 0.001, nepocas = 150, tol = 0.01):
        self.p = p
        self.η = η
        self.nepocas = nepocas        
        self.tol = tol
        self.h = np.tanh
        self.hx = sech2
        self.g = lambda x: x
        self.gx = lambda x: 1
        self.erros = dict()

    def fit(self, X_train, y_train):
        n, m = X_train.shape[1], 1
        self.z = np.random.normal(size = (n + 1, self.p))
        self.w = np.random.normal(size = (self.p + 1, m))
        self.erros.clear()
        for epoca in range(self.nepocas):
            for idx in np.random.permutation(len(X_train)):
                ui, uj = self.dotLayers(X_train[idx])
                e = y_train[idx] - self.g(uj)
                δ = e*self.gx(uj)

                for i in range(self.p):
                    for j in range(m):
                        self.w[i][j] += self.η*δ[j]*self.h(ui[i])
                
                for l in range(n):
                    for i in range(self.p):
                        self.z[l][i] += self.η*sum([δ[j]*self.w[i][j] for j in range(m)]) * self.hx(ui)[i] * X_train[idx][l]
                self.erros[epoca] = [e[0]**2] if epoca not in self.erros else self.erros[epoca] + [e[0]**2]

    def dotLayers(self, X):
        ui = np.dot(bias(X), self.z)
        uj = np.dot(bias(self.h(ui)), self.w)
        return ui, uj

    def predict(self, X, f = np.round):
        _, uj = self.dotLayers(X)
        return f(self.g(uj))

## [4] CLASSE PARA VISUALIZAÇÃO

In [ ]:
class GIF():
    def __init__(self, folder = 'data/img'):
        self.folder = folder

    def Preparar(self):
        if os.path.exists(self.folder):
            shutil.rmtree(self.folder)
        os.mkdir(self.folder)
    
    def FileNames(self):
        arquivos_info = []
        for nome_imagem in os.listdir(self.folder):
            caminho_completo = os.path.join(self.folder, nome_imagem)
            data_criacao = datetime.datetime.fromtimestamp(os.path.getctime(caminho_completo))
            arquivos_info.append((caminho_completo, data_criacao))
        arquivos_info.sort(key=lambda x: x[1])
        return np.array(arquivos_info)[:, 0]
    
    def Make(self, path = 'data'):
        imagens = []
        for caminho in self.FileNames():
            imagem = Image.open(caminho)
            imagens.append(imagem)
        imagens[0].save(f'{path}/gif.gif', save_all=True, append_images=imagens[1:], duration=400, loop=0)

## [5] FUNÇÃO PARA SUPERFÍCIE DE SEPARAÇÃO

In [ ]:
def Grid(Neuronio, ax, alpha = 0.5, palette = 'plasma'):
    eixo_x = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 200)
    eixo_y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 200)
    xx, yy = np.meshgrid(eixo_x, eixo_y)
    pontos = np.column_stack((xx.ravel(), yy.ravel()))
    labels_pred = Neuronio.predict(pontos).reshape(xx.shape).round(2)
    ax.contourf(xx, yy, labels_pred, alpha = alpha, cmap = palette)

***
# <font color=seagreen size=10>ATIVIDADE 5</font>

[ENUNCIADO]

<em>Para observar que o MLP é capaz de aproximar qualquer função contínua, deve realizar a
regressão de um ciclo de uma senoide MLP com backpropagation. A função de ativação
da camada de saída deve ser linear, e devem haver 3 ou mais neurônios na camada escondida.</em>

***

In [ ]:
GIF().Preparar()
metricas = []
for p in range(2, 101, 2):
    X = np.array([np.linspace(0, 2*np.pi, 200)]).T
    y = np.sin(X.ravel()+np.random.normal(scale=0.25, size=200))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    Rede = TwoLayersAdaline(p = p)
    Rede.fit(X_train, y_train)
    y_pred = Rede.predict(X_test, f = lambda x: x)

    metricas.append([r2_score(y_test, y_pred), 1 - mean_squared_error(y_test, y_pred)])
    df_metricas = lambda x: pd.DataFrame(metricas).iloc[:, x].round(2)
    labelr2  = f'R²        (x̄: {df_metricas(0).mean():.2f}, σ: {df_metricas(0).std():.2f})'
    labelmse = f'1 - MSE (x̄: {df_metricas(1).mean():.2f}, σ: {df_metricas(1).std():.2f})  '
    fig, axs = plt.subplots(1, 2, figsize = (14, 6))
    fig.suptitle(f'MultiLayer Perceptron: Aproximação da função sen(x) com {p} neurônios', fontsize = 16, y = 1.01)
    sns.scatterplot(x = X_train.ravel(), y = y_train.ravel(), label = 'Dados de Treino', ax = axs[0], s = 15)
    sns.scatterplot(x = X_test.ravel(), y = y_test.ravel(), label = 'Dados de Teste', ax = axs[0], s = 15)
    sns.lineplot(x = X_test.ravel(), y = y_pred.ravel(), label = 'Aproximação', color = 'forestgreen', ax = axs[0])
    sns.lineplot(x = [2*i+2 for i in range(len(metricas))], y = df_metricas(0), label = labelr2, ax = axs[1])
    sns.lineplot(x = [2*i+2 for i in range(len(metricas))], y = df_metricas(1), label = labelmse, ax = axs[1])
    axs[0].set_title('Resultado da Aproximação')
    axs[0].set_ylim(-1.5, 1.5)
    axs[0].set_xlabel('x1')
    axs[0].set_ylabel('y1')
    axs[1].set_title('Análise de Métricas')
    axs[1].set_ylim(0, 1.05)
    axs[1].set_xlabel('Quantidade de Neurônios')
    axs[1].set_ylabel('Métricas')
    axs[1].legend(loc='lower right')
    axs[1].set_xlim(0, p + 2)
    plt.savefig(f'data/img/{p}.png', bbox_inches = 'tight')
    plt.close()
    
GIF().Make()

<div align = 'center'>
    <img src = "data/gif.gif">
</div>

***
# <font color=seagreen size=10>ATIVIDADE 6</font>

***

## [1] PARTE 1

***
[ENUNCIADO]
<em>Utilizando a mesma rede MLP que você implementou para resolver o problema da aproximação da função senoidal, resolva o problema de classificação do OU-Exclusivo (XOR) mostrado na Figura 1. As classes devem ser como mostradas na figura (alternadas). O aluno deve:

1. Criar os dados de treinamento e teste.
2. Treinar uma RNA com uma camada escondida de 3 ou mais neurônios com função de ativação tangente hiperbólica e um neurônio de saída com função de ativação linear.
3. Testar o modelo obtido com os dados de teste (separe 30% dos dados aleatoriamente para teste). Lembre-se de definir um limiar para a classificação da saída.
4. Plotar a superfície de separação no espaço de entrada.
5. Calcular a acurácia de treino e teste. </em>
***

In [ ]:
dados = pd.read_csv('data/xor.csv')[['x1', 'x2', 'y']]
X = dados.drop(['y'], axis = 1).values
y = dados['y'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
Rede = TwoLayersAdaline(p=100, nepocas=500)
Rede.fit(X_train, y_train)
y_pred = Rede.predict(X_test)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (12, 4))
fig.suptitle('Rede MLP: Problema XOR com 100 Neurônios', y = 1.03)

sns.scatterplot(x = X_train[:, 0], y = X_train[:, 1], hue = y_train, palette='plasma', ax = axs[0])
axs[0].set_title('Dados de Treino')

sns.scatterplot(x = X_test[:, 0], y = X_test[:, 1], hue = y_test, palette='plasma', ax = axs[1])
axs[1].set_title(f'Dados de Teste (Acurácia: {score(y_test, y_pred):.2f})')
Grid(Rede, axs[1])

for ax in axs.ravel(): 
    ax.get_legend().remove()

<div align = 'center'>
    <img src = "data/Ex6-1.png">
</div>

## [2] PARTE 2
***
[ENUNCIADO]

<em>Aplique esta mesma rede ao problema do Câncer de mama (Breast Cancer). Esta base de dados pode ser carregada do pacote mlbench. Esta base de dados possui 9 variáveis de entrada, uma variável de saída com a classificação das 699 amostras em maligno e benigno. Nesta atividade, o aluno irá realizar o treinamento da MLP para separar as classes e avaliar o desempenho do mesmo. O aluno deverá então:

Carregar os dados e armazená-los. Estes dados devem receber um tratamento inicial para eliminação dos dados faltantes. Os dados faltantes são representados pelo string NaN.

1. Rotular as amostras das classes com o valor de 0 (maligno) e 1 (benigno).

2. Selecionar aleatoriamente 70% das amostras para o conjunto de treinamento e 30% para o conjunto de teste, para cada uma das duas classes.

3. Utilizar as amostras de treinamento para fazer o treinamento da MLP.

4. Aplicar o modelo treinado na classificação do conjunto de testes.

5. Calcular o erro percentual. (O erro é dado pelo número de amostras de teste classificadas de forma errada)

6. Criar um loop para repetir 10 vezes os itens 2-6 (Validação cruzada com 10 folds), armazenando o valor do erro percentual do item 

7. Calcular a acurácia média e seu desvio padrão.</em>
***
Nota: por estar realizando em Python, tive de buscar a base na internet, a partir do link: https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original

In [ ]:
dados = pd.read_csv('data/breast-cancer-wisconsin.csv')
dados = dados.replace('?', np.nan).dropna().astype(int)
dados['Class'] = dados['Class'].map({2: 1, 4: 0})

dados = dados.sample(frac=1)
X = dados.drop('Class', axis = 1).values
y = dados['Class'].values

In [ ]:
erros = dict()
for p in range(5, 25, 5):
    erros[p] = dict()
    for fold, (train_idx, test_idx) in enumerate(KFold(n_splits=10).split(X)):
        X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx] , y[test_idx] 
        Rede = TwoLayersAdaline(p=p, nepocas=200)
        Rede.fit(X_train, y_train)
        y_pred = Rede.predict(X_test)
        erros[p][fold] = score(y_test, y_pred) * 100

In [ ]:
df = pd.DataFrame(erros).iloc[:, ::-1]
ax = sns.boxplot(df, orient = 'h')
ax.figure.set_size_inches(12, 7)
ax.set_title('Desempenho da Rede (10 Folds)', fontsize = 18)
ax.set_xlabel('Acurácia (%)', fontsize = 16)
ax.set_ylabel('Quantidade de Neurônios', fontsize = 16)
ax.set_xlim(0, 100)
for i in range(4):
    ax.text(x = 10, y = i + 0.07, s = f' (Média: {df.mean().iloc[i]:.2f}, Desvio Padrão: {df.std().iloc[i]:.2f})', va ='bottom', ha= 'left')

<div align = 'center'>
    <img src = "data/Ex6-2.png">
</div>